In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import dill
import sys
sys.path.append("..")

from utils.helper import Tokenizer, preprocess_sentence

In [2]:
df1 = pd.read_csv("../Data/translation_train.csv",encoding='UTF-8')
df1.head(10)

,English,Arabic
0,I have been dealt four aces.,لقد رميت اربع اوراق من نوع أس.
1,rejoice over the verdict.,ابتهاج نتيجة اصدار الحكم
2,today the people of south africa marched in su...,اليوم شعب جنوب افريقيا يدعم الفلسطينيين الذين ...
3,earlier this month the ministry of health decl...,وقد صرحت وزارة الصحة باكرا هذا الشهر ان اكثر م...
4,sharek posted this video titled sharek partici...,نشر هذا الفيديو تحت عنوان شارك داعيا الشعب للا...
5,When was the castle built?,متى بُنيت القلعة؟
6,ha it s nice to be known as cool and calm it s...,اه من الجيد ان اكون معروفة باني هادية ومرتاحة ...
7,egypt muslim brotherhood trial postponed globa...,مصر تاجيل محاكمة اعضاء في الاخوان المسلمين الا...
8,I love rock.,أعشق موسيقى الروك.
9,This happened for a reason.,حدث هذا لسبب.


In [3]:
df2 = pd.read_csv("../Data/ara.txt", sep='\t',encoding='UTF-8',header=None).drop(columns=[2]).rename(columns={0:"English",1:"Arabic"})
df2.head(10)

,English,Arabic
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Duck!,اخفض رأسك!
3,Duck!,اخفضي رأسك!
4,Duck!,اخفضوا رؤوسكم!
5,Help!,النجدة!
6,Jump!,اقفز!
7,Stop!,قف!
8,Stop!,توقف !
9,Wait!,إنتظر


In [4]:
df = pd.concat([df1,df2],ignore_index=True)
df.head(10)

,English,Arabic
0,I have been dealt four aces.,لقد رميت اربع اوراق من نوع أس.
1,rejoice over the verdict.,ابتهاج نتيجة اصدار الحكم
2,today the people of south africa marched in su...,اليوم شعب جنوب افريقيا يدعم الفلسطينيين الذين ...
3,earlier this month the ministry of health decl...,وقد صرحت وزارة الصحة باكرا هذا الشهر ان اكثر م...
4,sharek posted this video titled sharek partici...,نشر هذا الفيديو تحت عنوان شارك داعيا الشعب للا...
5,When was the castle built?,متى بُنيت القلعة؟
6,ha it s nice to be known as cool and calm it s...,اه من الجيد ان اكون معروفة باني هادية ومرتاحة ...
7,egypt muslim brotherhood trial postponed globa...,مصر تاجيل محاكمة اعضاء في الاخوان المسلمين الا...
8,I love rock.,أعشق موسيقى الروك.
9,This happened for a reason.,حدث هذا لسبب.


In [5]:
df.duplicated().sum()

10029

In [6]:
df = df.drop_duplicates()

In [7]:
df.shape

(25569, 2)

In [8]:
df["English"] = df["English"].apply(lambda x: preprocess_sentence(x,"en"))
df["Arabic"] = df["Arabic"].apply(lambda x: preprocess_sentence(x,"ar"))

In [9]:
df.head(10)

,English,Arabic
0,<start> i have been dealt four aces . <end>,<start> لقد رميت اربع اوراق من نوع اس . <end>
1,<start> rejoice over the verdict . <end>,<start> ابتهاج نتيجة اصدار الحكم <end>
2,<start> today the people of south africa march...,<start> اليوم شعب جنوب افريقيا يدعم الفلسطينيي...
3,<start> earlier this month the ministry of hea...,<start> وقد صرحت وزارة الصحة باكرا هذا الشهر ا...
4,<start> sharek posted this video titled sharek...,<start> نشر هذا الفيديو تحت عنوان شارك داعيا ا...
5,<start> when was the castle built ? <end>,<start> متى بنيت القلعة؟ <end>
6,<start> ha it s nice to be known as cool and c...,<start> اه من الجيد ان اكون معروفة باني هادية ...
7,<start> egypt muslim brotherhood trial postpon...,<start> مصر تاجيل محاكمة اعضاء في الاخوان المس...
8,<start> i love rock . <end>,<start> اعشق موسيقى الروك . <end>
9,<start> this happened for a reason . <end>,<start> حدث هذا لسبب . <end>


In [10]:
if not os.path.exists("../Models/EN_Tokenizer.pkl"):
    EN = Tokenizer()
    EN.fit_on_texts(df["English"].values)
else:
    with open('../Models/EN_Tokenizer.pkl', 'rb') as file:
        EN = dill.load(file)

if not os.path.exists("../Models/AR_Tokenizer.pkl"):
    AR = Tokenizer()
    AR.fit_on_texts(df["Arabic"].values)
else: 
    with open('../Models/AR_Tokenizer.pkl', 'rb') as file:
        AR = dill.load(file)

In [11]:
df["English_seq"] = df["English"].apply(lambda x: EN.texts_to_sequences([x])[0])
EN.maxlen = df["English_seq"].apply(len).max()
df["Arabic_seq"] = df["Arabic"].apply(lambda x: AR.texts_to_sequences([x])[0])
AR.maxlen = df["Arabic_seq"].apply(len).max()
df.head()

,English,Arabic,English_seq,Arabic_seq
0,<start> i have been dealt four aces . <end>,<start> لقد رميت اربع اوراق من نوع اس . <end>,"[60, 10918, 10298, 2414, 5788, 8970, 293, 4, 59]","[39, 35901, 25960, 1429, 14269, 41831, 43883, ..."
1,<start> rejoice over the verdict . <end>,<start> ابتهاج نتيجة اصدار الحكم <end>,"[60, 19045, 16539, 22846, 24304, 4, 59]","[39, 97, 43052, 2465, 6731, 38]"
2,<start> today the people of south africa march...,<start> اليوم شعب جنوب افريقيا يدعم الفلسطينيي...,"[60, 23099, 22846, 17059, 16132, 21481, 572, 1...","[39, 13333, 28503, 23295, 3160, 52263, 9389, 7..."
3,<start> earlier this month the ministry of hea...,<start> وقد صرحت وزارة الصحة باكرا هذا الشهر ا...,"[60, 7050, 22917, 14981, 22846, 14724, 16132, ...","[39, 49647, 29038, 48766, 8307, 15199, 44177, ..."
4,<start> sharek posted this video titled sharek...,<start> نشر هذا الفيديو تحت عنوان شارك داعيا ا...,"[60, 20709, 17685, 22917, 24379, 23075, 20709,...","[39, 43363, 44177, 9463, 19915, 30423, 28218, ..."


In [13]:
df["English_seq"] = df["English_seq"].apply(lambda x: "_".join(np.array(x).astype(str)))
df["Arabic_seq"] = df["Arabic_seq"].apply(lambda x: "_".join(np.array(x).astype(str)))

In [14]:
with open('../Models/EN_Tokenizer.pkl', 'wb') as file:
    dill.dump(EN, file)

with open('../Models/AR_Tokenizer.pkl', 'wb') as file:
    dill.dump(AR, file)

In [15]:
df[["English_seq","Arabic_seq"]].to_csv("../Data/data.csv",index=False)